In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def build_model(labels, convolutional_layers=2, pooling_layers=True, input_shape=None):
    """
    Creates a simple convolutional network by adding layers based on input
    
    Parameters:
        labels: int
            Number of layers for prediction (i.e. size of output layer of the model). Should be positive and higher than 1.
        convolutional_layers: int (default 2)
            Number of convolutional layers (using tensorflows Conv2D, including the input convolutional layer). Should be positive.
        pooling_layers: bool (default True)
            Whether to include pooling layers between every pair of convolutional layers
        input_shape: (int, int, int) or (int, int) or None (default None):
            Shape of the image that will be given as input (i.e. input shape of the first layer).
            Integers in the tuple are required to be positive.

    Returns:
        model: keras.engine.sequential.Sequential
            Model created according to input
    """
    # Input management
    if not isinstance(labels, int):
        raise ValueError("Different datatype than integer has been given as input for the number of labels")
    
    if not isinstance(convolutional_layers, int):
        raise ValueError("Different datatype than integer has been given as input for the number of convolutional layers")

    if labels < 1:
        raise ValueError("Number of labels is less than 1. Please specify different amount.")

    if labels == 1:
        wrn = "\nYou have entered 1 as the number of labels.\n" 
        wrn += "This might result in unpredicted behaviour and there is not much point in building a model then"
        warnings.warn(wrn)

    if convolutional_layers < 1:
        raise ValueError("This function expects at least one convolutional layer to be present in the model.")

    if not isinstance(pooling_layers, bool):
        raise ValueError("Different datatype than boolean has been given as input for the pooling_layers parameter")

    if input_shape:
        if not isinstance(input_shape, tuple):
            raise ValueError("Input shape has been assigned and different input than tuple was given")
        if len(input_shape) not in [2, 3]:
            raise ValueError("2D or 3D images expected as input")
        elif len(input_shape) == 2:
            input_shape = tuple([*input_shape, 1])
        for val in input_shape:
            if not isinstance(val, int):
                raise ValueError("Integers were expected in place of image dimensions in parameter input_shape")
            if val < 0:
                raise ValueError("One of the dimensions of the input shape given is negative. Please give correct input shape.")

    model = models.Sequential()

    if input_shape:
        model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=input_shape))
    else:
        model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    for _ in range(convolutional_layers - 1):
        if pooling_layers:
            model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(labels))
    
    return model

In [ ]:
script_dir = os.path.dirname("Image Collection.ipynb")
data_dir = os.path.join(script_dir, "Data")

In [ ]:
gestures = ["I", "My", "You", "Your", 
            "In", "To", "With", "Yes", 
            "No", "Well", "I love you",
            "Oh I see", "Name", "Hug",
            "Internet", "Bus", "Money",
            "Work", "Ask", "Go",
            "Look", "Have", "Correct",
            "Want", "Where", 
            "A", "B", "C", "D", 
            "E", "F", "G", "H", 
            "I", "K", "L", "M", 
            "N", "O", "P", "Q", 
            "R", "S", "T", "U", 
            "V", "W", "X", "Y"]

In [ ]:
train_images = tf.keras.preprocessing.image_dataset_from_directory(data_dir,
                                                                   validation_split=0.25,
                                                                  subset="training",
                                                                  seed=123,
                                                                  image_size=(128, 128),
                                                                  color_mode="grayscale")

In [ ]:
test_images = tf.keras.preprocessing.image_dataset_from_directory(data_dir,
                                                                   validation_split=0.25,
                                                                  subset="validation",
                                                                  seed=123,
                                                                  image_size=(128, 128),
                                                                  color_mode="grayscale")

In [ ]:
model = build_model(labels=49, convolutional_layers=2, input_shape=(128, 128))

In [ ]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

history = model.fit(train_images, batch_size=200, epochs=11, 
                    validation_data=(test_images), steps_per_epoch=50)

In [ ]:
import numpy as np
import cv2

im = cv2.imread("Data/Yes/Yes_88.jpg", cv2.IMREAD_GRAYSCALE)
im = np.expand_dims(im, axis=0)
im = np.expand_dims(im, axis=-1)
prediction = model(im, training=False)
os.listdir("Data")[np.argmax(prediction, axis = 1)[0]]

In [ ]:
model.summary()

In [ ]:
model.load_weights("Weights/weights")

In [ ]:
im = cv2.imread("Data/Yes/Yes_88.jpg", cv2.IMREAD_GRAYSCALE)
im = np.expand_dims(im, axis=0)
im = np.expand_dims(im, axis=-1)
prediction = model(im, training=False)
os.listdir("Data")[np.argmax(prediction, axis = 1)[0]]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
figure = fig.get_figure()    
figure.savefig("training.png", dpi=400) 

In [ ]:
df = pd.DataFrame(history.history)
df.drop(columns=["val_loss", "loss"], inplace=True)

plt.figure(figsize=(15, 10), facecolor=(1, 1, 1))

fig = sb.lineplot(data=df, sizes=[1.7, 1.7])
plt.title("Progression of model performance with each epoch", fontsize=20)
plt.xlabel("Number of epochs", fontsize=16)
plt.xticks(range(0, 11), fontsize=14)
plt.ylabel("Accuracy", fontsize=16)
plt.yticks(np.arange(0, 1.1, 0.1), fontsize=14)
plt.legend(["Training accuracy", "Validation accuracy"], fontsize=14)

plt.show()

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()

In [ ]:
model.save_weights("Weights/weights")

In [ ]:
#import numpy as np
#import cv2

#im = cv2.imread("Data/Yes/Yes_2.jpg", cv2.IMREAD_GRAYSCALE)
#im = np.expand_dims(im, axis=0)
#im = np.expand_dims(im, axis=-1)
#prediction = model(im, training=False)
#test_images.class_names[np.argmax(prediction, axis = 1)[0]]

<h1>Building data pipeline</h1>

In [4]:
# Set threading options to autotuning
tf.config.threading.set_inter_op_parallelism_threads(0)
tf.config.threading.set_intra_op_parallelism_threads(0)

In [5]:
import json
import time

In [6]:
with open("config.json", "r") as file:
    config = json.load(file)

In [7]:
config

{'Paths': {'Data pipelines': 'data_pipelines',
  'Logs': 'logs',
  'Translations': 'translations.txt',
  'Gesture list': 'gestures.txt'},
 'Model': {'Current model': 'model/current',
  'Model experiments': 'model/experiments'},
 'General parameters': {'Image size': 196,
  'Desired amount': 100,
  'Top-up amount': 100}}

In [ ]:
# TODO: Specify validation split size in config
# TODO: Specify batch size in config

In [ ]:
# Note that image_dataset_from_directory shuffles dataset by default during loading (can be turned off)
    # Source: https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(config["Paths"]["Data"],
                                                       validation_split=0.25,
                                                       subset="training",
                                                       seed=123,
                                                       label_mode="categorical",
                                                       image_size=(config["General parameters"]["Image size"],
                                                                   config["General parameters"]["Image size"]),
                                                       batch_size=200)

Found 490 files belonging to 49 classes.
Using 368 files for training.


In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(config["Paths"]["Data"],
                                                      validation_split=0.25,
                                                      subset="train",
                                                      seed=123,
                                                      label_mode="categorical",
                                                      image_size=(config["General parameters"]["Image size"],
                                                                  config["General parameters"]["Image size"]),
                                                      batch_size=200)

In [9]:
with open(config["Paths"]["Gesture list"], "r") as gesture_list:
    gestures = gesture_list.readlines()[0].split(", ")

In [10]:
print(gestures)

['I index', 'My', 'You', 'Your', 'In', 'To', 'With', 'Yes', 'No', 'Well', 'I love you', 'Oh I see', 'Name', 'Hug', 'Internet', 'Bus', 'Money', 'Work', 'Ask', 'Go', 'Look', 'Have', 'Correct', 'Want', 'Where', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


In [46]:
AUTOTUNE = tf.data.AUTOTUNE
train_images, test_images = tf.keras.preprocessing.image_dataset_from_directory("Data",
                                                                                labels="inferred",
                                                                                label_mode="categorical",
                                                                                class_names=gestures,
                                                                                batch_size=32,
                                                                                image_size=(config["General parameters"]["Image size"],
                                                                                            config["General parameters"]["Image size"]),
                                                                                shuffle=True,
                                                                                seed=123,
                                                                                validation_split=0.2,
                                                                                subset="both")

Found 490 files belonging to 49 classes.
Using 392 files for training.
Using 98 files for validation.


In [47]:
train_images = train_images.cache().prefetch(buffer_size=AUTOTUNE)
test_images = test_images.cache().prefetch(buffer_size=AUTOTUNE)

In [17]:
from tensorflow.keras import layers, Sequential
from model.preprocessing import AdaptiveThresholding, Blurring, Grayscale

In [33]:
pipeline1 = Sequential(
            [
                Grayscale(),
                Blurring(blurring_type="median", kernel_size=3, sigma=None),
                AdaptiveThresholding(thresholding_type="mean", block_size=3, constant=0),
                layers.Rescaling(scale=(1. / 255))
            ]
        )

In [88]:
def create_model():
    inputs = tf.keras.Input(shape=(196, 196, 3))
    x = Grayscale()(inputs)
    x = Blurring()(x)
    x = AdaptiveThresholding()(x)
    x = tf.keras.layers.Rescaling(scale=1./255)(x)
    x = tf.keras.layers.Conv2D(16, 3, activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    outputs = tf.keras.layers.Dense(49, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [89]:
full_model = create_model()

TypeError: 'NoneType' object is not subscriptable

In [77]:
full_model.compile(optimizer="adam",
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=["accuracy"])

In [78]:
full_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 196, 196, 3)]     0         
                                                                 
 grayscale_17 (Grayscale)    (None, 196, 196, 1)       0         
                                                                 
 rescaling_15 (Rescaling)    (None, 196, 196, 1)       0         
                                                                 
 lambda_1 (Lambda)           (None, 196, 196, 1)       0         
                                                                 
 conv2d_20 (Conv2D)          (None, 194, 194, 16)      160       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 97, 97, 16)       0         
 g2D)                                                            
                                                           

In [79]:
history = full_model.fit(train_images, epochs=1, 
                    validation_data=(test_images))

13/13 [==============================] - 5s 340ms/step - loss: 4.2616 - accuracy: 0.0230 - val_loss: 3.8473 - val_accuracy: 0.0408


In [ ]:
list_ds = tf.data.Dataset.list_files(str("Old_data/data_64x64/*/*"))

In [ ]:
for f in list_ds.take(5):
    print(f.numpy())

In [ ]:
# Define a function to load images and extract labels
def process_paths(dir_name):
    label = tf.strings.split(dir_name, os.sep)[-2]
    return tf.io.read_file(dir_name), label

In [ ]:
labeled_ds = list_ds.map(process_paths)

In [ ]:
for image_raw, label_text in labeled_ds.take(1):
    print(repr(image_raw.numpy()[:10]))
    print()
    print(label_text.numpy())

In [ ]:
# While large buffer_sizes shuffle more thoroughly,
# they can take a lot of memory, and significant time to fill.
# (quote: tensorflow.org/guide/data)
labeled_ds = labeled_ds.shuffle(buffer_size=1000)

In [ ]:
# Batch dataset and get full element shape propagation using drop_remainder=True
batched_dataset = labeled_ds.batch(100, drop_remainder=True)

In [ ]:
for batch in batched_dataset.take(1):
    print([sample.numpy()[:1] for sample in batch])

In [ ]:
# Dataset.m